<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#RentListingInquries" data-toc-modified-id="RentListingInquries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>RentListingInquries</a></span><ul class="toc-item"><li><span><a href="#基本信息" data-toc-modified-id="基本信息-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>基本信息</a></span></li><li><span><a href="#导入工具包" data-toc-modified-id="导入工具包-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>导入工具包</a></span></li><li><span><a href="#导入数据" data-toc-modified-id="导入数据-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>导入数据</a></span></li><li><span><a href="#分割数据" data-toc-modified-id="分割数据-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>分割数据</a></span></li><li><span><a href="#数据量太大,分割数据" data-toc-modified-id="数据量太大,分割数据-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>数据量太大,分割数据</a></span></li></ul></li><li><span><a href="#模型选择" data-toc-modified-id="模型选择-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>模型选择</a></span><ul class="toc-item"><li><span><a href="#DecisionTreeClassifier" data-toc-modified-id="DecisionTreeClassifier-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>DecisionTreeClassifier</a></span></li><li><span><a href="#RandomForest" data-toc-modified-id="RandomForest-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>RandomForest</a></span></li></ul></li><li><span><a href="#预测保存" data-toc-modified-id="预测保存-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>预测保存</a></span></li></ul></div>

# RentListingInquries

## 基本信息
bathrooms: 浴室的数量number of bathrooms
bedrooms: 卧室数量number of bedrooms
building_id: ID号
created: 
description: 描述
display_address: 
features: 关于这间公寓的功能列表a list of features about this apartment
latitude: 纬度
listing_id
longitude: 经度
manager_id
photos: 照片链接列表a list of photo links. You are welcome to download the pictures yourselves from renthop's site, but they are the same as imgs.zip. 
price: 以美元计算in USD
street_address: 街道地址
interest_level: 这是目标变量。它有3个类别：'高'，'中等'，'低'this is the target variable. It has 3 categories: 'high', 'medium', 'low'

## 导入工具包

In [1]:
# 数据导入与处理
import numpy as np
import pandas as pd
# 数据可视化
import matplotlib.pyplot as plt
import seaborn as sns


## 导入数据

In [2]:
# 课程网站提供的特征工程编码后的数据（RentListingInquries_FE_train.csv）
# 或稀疏编码的形式（RentListingInquries_FE_train.bin）
# 读取数据
# path to where the data lies
dpath = './data/'
train = pd.read_csv(dpath +"train_FE.csv")
train.head()

,bathrooms,bedrooms,price,price_bathrooms,price_bedrooms,room_diff,room_num,Year,Month,Day,...,walk,walls,war,washer,water,wheelchair,wifi,windows,work,interest_level
0,1.5,3,3000,1200.0,750.000000,-1.5,4.5,2016,6,24,...,0,0,0,0,0,0,0,0,0,1
1,1.0,2,5465,2732.5,1821.666667,-1.0,3.0,2016,6,12,...,0,0,0,0,0,0,0,0,0,2
2,1.0,1,2850,1425.0,1425.000000,0.0,2.0,2016,4,17,...,0,0,0,0,0,0,0,0,0,0
3,1.0,1,3275,1637.5,1637.500000,0.0,2.0,2016,4,18,...,0,0,0,0,0,0,0,0,0,2
4,1.0,4,3350,1675.0,670.000000,-3.0,5.0,2016,4,28,...,0,0,1,0,0,0,0,0,0,2


In [3]:
# 测试数据
test = pd.read_csv(dpath+'test_FE.csv')
test.head()

,bathrooms,bedrooms,price,price_bathrooms,price_bedrooms,room_diff,room_num,Year,Month,Day,...,virtual,walk,walls,war,washer,water,wheelchair,wifi,windows,work
0,1.0,1,2950,1475.000000,1475.000000,0.0,2.0,2016,6,11,...,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2850,1425.000000,950.000000,-1.0,3.0,2016,6,24,...,0,0,0,1,0,0,0,0,0,0
2,1.0,1,3758,1879.000000,1879.000000,0.0,2.0,2016,6,3,...,0,0,0,0,0,0,0,0,0,0
3,1.0,2,3300,1650.000000,1100.000000,-1.0,3.0,2016,6,11,...,0,0,0,0,0,0,1,0,0,0
4,2.0,2,4900,1633.333333,1633.333333,0.0,4.0,2016,4,12,...,0,0,0,1,0,0,0,0,0,0


In [4]:
train.isnull().sum().sum()

0

In [5]:
test.isnull().sum().sum()

0

## 分割数据

## 数据量太大,分割数据
train = train.sample(frac=0.3)
train.shape

In [6]:
# 区分测试集与训练集
X = train.drop('interest_level', axis=1)
y = train['interest_level']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y
    , test_size=0.2
    , random_state=33
    , stratify=y)

In [7]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39481 entries, 14659 to 25927
Columns: 226 entries, bathrooms to work
dtypes: float64(8), int64(218)
memory usage: 68.4 MB


# 模型选择

## DecisionTreeClassifier

In [8]:
# 分类评分标准
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import accuracy_score

In [9]:
from sklearn.tree import DecisionTreeClassifier
model_tree = DecisionTreeClassifier()

In [10]:
model_tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [11]:
y_pd = model_tree.predict(X_test)
model_tree.score(X_test, y_test)

0.6533279303008813

In [12]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pd))

             precision    recall  f1-score   support

          0       0.30      0.31      0.31       768
          1       0.35      0.35      0.35      2246
          2       0.79      0.79      0.79      6857

avg / total       0.65      0.65      0.65      9871



print(roc_auc_score(y_test,y_pdpbed))

缺省的决策树正确率为0.65

In [13]:
#查看特征重要性
df = pd.DataFrame({"columns":list(X_train.columns), 
                   "importance":list(model_tree.feature_importances_.T)})
df.sort_values(by=['importance'],ascending=False)

,columns,importance
4,price_bedrooms,0.107298
25,display_address_pred_1,0.085401
23,distance,0.079904
24,display_address_pred_0,0.072541
3,price_bathrooms,0.070777
11,Yday,0.065071
2,price,0.064622
9,Day,0.064459
12,hour,0.063819
10,Wday,0.037424


有好多的特征都起到作用了,估计树的深度较深

调节参数试试

In [14]:
from sklearn.tree import DecisionTreeClassifier

model_DD = DecisionTreeClassifier()

max_depth = range(1,10,1)
min_samples_leaf = range(1,10,2)

tuned_parameters = dict(max_depth=max_depth, min_samples_leaf=min_samples_leaf)
tuned_parameters

{'max_depth': range(1, 10), 'min_samples_leaf': range(1, 10, 2)}

In [17]:
from sklearn.model_selection import GridSearchCV
DD = GridSearchCV(model_DD, tuned_parameters, 
                  cv=5, 
                  n_jobs=-1, 
                  return_train_score=True)
DD.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': range(1, 10), 'min_samples_leaf': range(1, 10, 2)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [18]:
print("Best: %f using %s" % (DD.best_score_, DD.best_params_))

Best: 0.711076 using {'max_depth': 8, 'min_samples_leaf': 9}


In [19]:
y_pd = DD.predict(X_test)
model_tree.score(X_test, y_test)

0.6533279303008813

In [20]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pd))

             precision    recall  f1-score   support

          0       0.50      0.16      0.24       768
          1       0.43      0.24      0.30      2246
          2       0.76      0.93      0.84      6857

avg / total       0.66      0.71      0.67      9871



## RandomForest

In [21]:
from sklearn.ensemble import RandomForestClassifier

model_RR=RandomForestClassifier()

In [22]:
model_RR.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [23]:
y_pred = model_RR.predict(X_test)
model_RR.score(X_test, y_test)

0.7096545436126026

In [24]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.49      0.26      0.34       768
          1       0.42      0.33      0.37      2246
          2       0.79      0.89      0.83      6857

avg / total       0.68      0.71      0.69      9871



随机森林的得分也是0.709,高了不少nice

试试调参, min_samples_leaf and n_estimators

In [25]:
from sklearn.ensemble import RandomForestClassifier

model_RR=RandomForestClassifier()

tuned_parameters = {'min_samples_leaf': range(1,10,2), 'n_estimators' : range(1,10,2) }
#tuned_parameters = {'min_samples_leaf': range(9,19,2), 'n_estimators' : range(6,10,1) }

In [26]:
from sklearn.model_selection import GridSearchCV
RR = GridSearchCV(model_RR, tuned_parameters,
                  cv=5, 
                  n_jobs=-1, 
                  return_train_score=True)

In [27]:
RR.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'min_samples_leaf': range(1, 10, 2), 'n_estimators': range(1, 10, 2)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [29]:
RR.cv_results_

{'mean_fit_time': array([0.17945409, 0.44519176, 0.70838866, 0.91817985, 1.14517803,
        0.24151549, 0.36334443, 0.58671727, 0.78149467, 0.96275296,
        0.23484254, 0.34213233, 0.51997175, 0.72399688, 0.88655272,
        0.22522378, 0.30472035, 0.50820651, 0.68029556, 0.83602123,
        0.22364526, 0.30745034, 0.48623242, 0.63654485, 0.76003809]),
 'std_fit_time': array([0.01793578, 0.05140506, 0.01792125, 0.02079121, 0.02521184,
        0.04406023, 0.02704062, 0.02586538, 0.02139807, 0.02150381,
        0.0463596 , 0.04039406, 0.00795782, 0.06065839, 0.06833171,
        0.0341025 , 0.03516639, 0.02834085, 0.0153527 , 0.04429091,
        0.02931851, 0.02556937, 0.03497612, 0.04183606, 0.08529855]),
 'mean_score_time': array([0.01720328, 0.0249269 , 0.03286037, 0.04248724, 0.04785743,
        0.01736727, 0.02187343, 0.03178134, 0.03619418, 0.04054017,
        0.0153903 , 0.02179384, 0.03655071, 0.04295592, 0.03904266,
        0.01561389, 0.02026825, 0.03011942, 0.03515377, 0.03

In [30]:
print(RR.best_score_)
print(RR.best_params_)

0.7192573643018161
{'min_samples_leaf': 3, 'n_estimators': 9}


In [31]:
y_pred = RR.predict(X_test)
RR.score(X_test, y_test)

0.7215074460540979

In [32]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.59      0.17      0.26       768
          1       0.46      0.23      0.31      2246
          2       0.76      0.94      0.84      6857

avg / total       0.68      0.72      0.67      9871



调参后得分也是0.72, min_samples_leaf=3, n_estimators=9

# 预测保存

In [33]:
# 选用调参后的随机森林模型生成数据 
y_ptrdict = RR.predict(test)

In [34]:
submission = test.loc[:,['Year','Month', 'Day', 'Wday', 'Yday', 'hour']]
#submission

In [36]:
submission['interest_level']= y_ptrdict
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74659 entries, 0 to 74658
Data columns (total 7 columns):
Year              74659 non-null int64
Month             74659 non-null int64
Day               74659 non-null int64
Wday              74659 non-null int64
Yday              74659 non-null int64
hour              74659 non-null int64
interest_level    74659 non-null int64
dtypes: int64(7)
memory usage: 4.0 MB


In [38]:
submission.to_csv(dpath+'submission_crat.csv')